<a href="https://colab.research.google.com/github/BetikuOluwatobi/named_entity_recognition_with_spacy/blob/main/named_entity_recognition_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Connect to reddit api and get access token

### You can get your client_id by creating an app here [Create app](https://www.reddit.com/prefs/apps).
[Read More](https://stateful.com/blog/reddit-oauth)

In [1]:
import requests

In [2]:
auth_tok = {"client_id":"your_app_client_id", "secret":"your_app_secret_key", "password":"your_reddit_password"}

In [3]:
auth = requests.auth.HTTPBasicAuth(auth_tok['client_id'],auth_tok['secret'])

In [4]:
type(auth)

requests.auth.HTTPBasicAuth

In [5]:
data = {'grant_type': 'password',
        'username': 'your_reddit_username',
        'password': auth_tok['password']}
headers = {'User-Agent': "app_name/0.0.1"}

In [6]:
access = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers)

In [7]:
headers['Authorization'] = f"bearer {access.json()['access_token']}"

### Get the investing subreddit data

In [9]:
api = "https://oauth.reddit.com/"
res = requests.get(f"{api}/r/investing/new", headers=headers, params={'limit': '100'})

In [10]:
res.json()['data']["children"]

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'investing',
   'selftext': 'Have a general question?  Want to offer some commentary on markets?  Maybe you would just like to throw out a neat fact that doesn\'t warrant a self post?  Feel free to post here! \n\nIf your question is "I have $10,000, what do I do?" or other "advice for my personal situation" questions, you should include relevant information, such as the following:\n\n* How old are you? What country do you live in?  \n* Are you employed/making income? How much?  \n* What are your objectives with this money? (Buy a house? Retirement savings?)  \n* What is your time horizon? Do you need this money next month? Next 20yrs?  \n* What is your risk tolerance? (Do you mind risking it at blackjack or do you need to know its 100% safe?)  \n* What are you current holdings? (Do you already have exposure to specific funds and sectors? Any other assets?)  \n* Any big debts (include interest rate) or expenses?  \n* An

In [11]:
len(res.json()['data']["children"])

100

### Create a dataframe for the pulled data

In [12]:
import pandas as pd
data = pd.DataFrame({'name':[], 'created_utc':[],
                     'subreddit':[],'selftext':[],
                     'title':[],'upvote_ratio':[],
                     'ups':[],'downs':[],'score':[],
                     'url':[]})
for post in res.json()['data']['children']:
    row = { 'name':post['data']['name'], 'created_utc':post['data']['created_utc'], 'subreddit':post['data']['subreddit'],'selftext':post['data']['selftext'],'title':post['data']['title'],
       'upvote_ratio':post['data']['upvote_ratio'],'ups':post['data']['ups'],'downs':post['data']['downs'],'score':post['data']['score'],'url':post['data']['url']}
    data = pd.concat([data, pd.DataFrame(row,index=[0])], ignore_index=True)

In [13]:
data

,name,created_utc,subreddit,selftext,title,upvote_ratio,ups,downs,score,url
0,t3_15eaqnj,1.690794e+09,investing,Have a general question? Want to offer some c...,Daily General Discussion and Advice Thread - J...,1.00,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/15...
1,t3_15e5my9,1.690776e+09,investing,New to the group and not sure if this question...,BRICS Currency Introduction - What's the best ...,0.45,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/15...
2,t3_15e5kpw,1.690776e+09,investing,With corporations trying to hoard homes to for...,How would a nation of renters affect retirement?,0.64,6.0,0.0,6.0,https://www.reddit.com/r/investing/comments/15...
3,t3_15e51vu,1.690775e+09,investing,Hello I am a 34yr old fully employed male look...,Selling my old home to pay for my new home,0.60,2.0,0.0,2.0,https://www.reddit.com/r/investing/comments/15...
4,t3_15e44fe,1.690772e+09,investing,I recently turned 29 and have felt skeptic abo...,Starting late and I have a boat load in saving...,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/15...
...,...,...,...,...,...,...,...,...,...,...
95,t3_15b7ikz,1.690478e+09,investing,Trying to figure out if I should be reinvestin...,What would you do? Drip or Buy,0.67,2.0,0.0,2.0,https://www.reddit.com/r/investing/comments/15...
96,t3_15b7g2i,1.690478e+09,investing,"I make less than $153,000 a year and have been...",Will getting married affect my Roth IRA?,0.90,116.0,0.0,116.0,https://www.reddit.com/r/investing/comments/15...
97,t3_15b6f6b,1.690475e+09,investing,We file jointly. Our income is too high for Ro...,Backdoor Roth tax intricacies?,0.77,25.0,0.0,25.0,https://www.reddit.com/r/investing/comments/15...
98,t3_15b6868,1.690475e+09,investing,Hey everyone -\n\nYesterday morning I read tha...,Explanation for huge FCF differences between a...,0.56,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/15...


### Pull more data

In [14]:
while True:
    res = requests.get(f"{api}/r/investing/new", headers=headers, params={'limit': '100', 'after': data['name'].iloc[-1]})
    if len(res.json()['data']['children']) > 0:
        for post in res.json()['data']['children']:
            row = { 'name':post['data']['name'], 'created_utc':post['data']['created_utc'], 'subreddit':post['data']['subreddit'],'selftext':post['data']['selftext'],'title':post['data']['title'],
               'upvote_ratio':post['data']['upvote_ratio'],'ups':post['data']['ups'],'downs':post['data']['downs'],'score':post['data']['score'],'url':post['data']['url']}
            data = pd.concat([data, pd.DataFrame(row,index=[0])], ignore_index=True)
    else:
        break

In [15]:
len(data)

926

In [16]:
data = data.replace({'|': ''}, regex=True)

In [17]:
data

,name,created_utc,subreddit,selftext,title,upvote_ratio,ups,downs,score,url
0,t3_15eaqnj,1.690794e+09,investing,Have a general question? Want to offer some c...,Daily General Discussion and Advice Thread - J...,1.00,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/15...
1,t3_15e5my9,1.690776e+09,investing,New to the group and not sure if this question...,BRICS Currency Introduction - What's the best ...,0.45,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/15...
2,t3_15e5kpw,1.690776e+09,investing,With corporations trying to hoard homes to for...,How would a nation of renters affect retirement?,0.64,6.0,0.0,6.0,https://www.reddit.com/r/investing/comments/15...
3,t3_15e51vu,1.690775e+09,investing,Hello I am a 34yr old fully employed male look...,Selling my old home to pay for my new home,0.60,2.0,0.0,2.0,https://www.reddit.com/r/investing/comments/15...
4,t3_15e44fe,1.690772e+09,investing,I recently turned 29 and have felt skeptic abo...,Starting late and I have a boat load in saving...,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/15...
...,...,...,...,...,...,...,...,...,...,...
921,t3_14afr25,1.686869e+09,investing,"In 2017, my employer started an ESOP program a...",Can't cash out my employee stock until I'm 73?,0.50,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...
922,t3_14afqlk,1.686869e+09,investing,I'm using a source where it doesn't calculate...,Confused with the Adjusted Closing Price for C...,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/14...
923,t3_14af1r3,1.686867e+09,investing,Obviously this comes down to personal risk app...,Are emergency funds really necessary?,0.22,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...
924,t3_14adyi0,1.686864e+09,investing,"Hi there,\n\nI recently bought long-bonds when...",Bond outlook when CB drop rates?,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/14...


##### You can save the text data this way

In [18]:
# data.to_csv("./reddit_investing.csv", sep="|", index=False)

### Extract Entities from text

In [19]:
#Import all necessary libraries
import pandas as pd
import numpy as np
import spacy
from spacy import displacy

In [20]:
#If you saved data you can read in the pulled data from investing subdreddit
#data = pd.read_csv('reddit_investing.csv',sep='|')

#### download spacy model for named_entity_recognition

In [21]:
!python -m spacy download en_core_web_sm

2023-07-31 09:58:08.177252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
nlp = spacy.load('en_core_web_sm')

#### Function to extract entity(organization) from text

In [23]:
def extract_entity(text):
    doc = nlp(text)
    org_list = []
    for ent in doc.ents:
        if ent.label_== "ORG":
            org_list.append(ent.text)
    org_list = list(set(org_list))
    return org_list

In [24]:
data["organizations"] = data["selftext"].apply(extract_entity)

In [25]:
data['organizations']

0                                      [wiki, FAQ]
1                               [the BRICS Summit]
2                                               []
3                                               []
4                                            [VOO]
                          ...                     
921                                         [ESOP]
922    [Yahoo Finance, the Adjusted Closing Price]
923                                             []
924                                             []
925                                 [Call Options]
Name: organizations, Length: 926, dtype: object

### Create a dictionary count of all organizations

In [26]:
from collections import Counter
orgs = [org for l in data['organizations'].tolist() for org in l]

In [27]:
orgs_list = Counter(orgs)

In [28]:
orgs_list.most_common(20)

[('VOO', 54),
 ('wiki', 46),
 ('FAQ', 46),
 ('Fidelity', 39),
 ('Vanguard', 18),
 ('SPY', 18),
 ('QQQ', 17),
 ('Fed', 15),
 ('fidelity', 11),
 ('AI', 11),
 ('Apple', 10),
 ('HSA', 10),
 ('MSFT', 9),
 ('Tesla', 9),
 ('Google', 8),
 ('fed', 8),
 ('ETF', 8),
 ('treasury', 8),
 ('IRS', 7),
 ('Treasury', 7)]

### Remove non-organizations by specifying black-list

In [29]:
def extract_entity(text):
    doc = nlp(text)
    org_list = []
    black_list = ['FAQ','HSA','AI', 'IRS', 'SEC', 'ETF', 'TREASURY', 'LLC', 'ISA','ROTH', 'COVID','EU','EUR','BND','STATE','EV',]
    for ent in doc.ents:
        if (ent.label_== "ORG" and ent.text.upper() not in black_list):
            org_list.append(ent.text.upper())
    org_list = list(set(org_list))
    return org_list

In [30]:
data["organizations"] = data["selftext"].apply(extract_entity)

In [31]:
data['organizations']

0                                           [WIKI]
1                               [THE BRICS SUMMIT]
2                                               []
3                                               []
4                                            [VOO]
                          ...                     
921                                         [ESOP]
922    [YAHOO FINANCE, THE ADJUSTED CLOSING PRICE]
923                                             []
924                                             []
925                                 [CALL OPTIONS]
Name: organizations, Length: 926, dtype: object

In [32]:
from collections import Counter
orgs = [org for l in data['organizations'].tolist() for org in l]

In [33]:
orgs_list = Counter(orgs)

In [34]:
orgs_list.most_common(20)

[('VOO', 54),
 ('WIKI', 46),
 ('FIDELITY', 46),
 ('FED', 23),
 ('VANGUARD', 18),
 ('SPY', 18),
 ('QQQ', 17),
 ('APPLE', 10),
 ('META', 9),
 ('MSFT', 9),
 ('TESLA', 9),
 ('GOOGLE', 8),
 ('NASDAQ', 8),
 ('MICROSOFT', 6),
 ('SPAXX', 6),
 ('SGOV', 6),
 ('FDIC', 6),
 ('NAV', 5),
 ('AMAZON', 5),
 ('NVIDIA', 5)]

### Install flair for sentence classification

In [35]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 51.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [36]:
import pandas as pd
import flair

In [37]:
model = flair.models.TextClassifier.load('en-sentiment')

2023-07-31 10:00:38,656 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpmhl1mmgs


100%|██████████| 253M/253M [00:32<00:00, 8.15MB/s]

2023-07-31 10:01:11,693 copying /tmp/tmpmhl1mmgs to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-07-31 10:01:12,282 removing temp file /tmp/tmpmhl1mmgs


### Function to create sentiment column for dataframe

In [38]:
def get_sentiment(text):
  sentence = flair.data.Sentence(text)
  model.predict(sentence)
  return (sentence.tag, sentence.score)

In [39]:
data["sentiment"] = data["selftext"].apply(get_sentiment)

In [40]:
data.tail(10)

,name,created_utc,subreddit,selftext,title,upvote_ratio,ups,downs,score,url,organizations,sentiment
916,t3_14amej9,1.686888e+09,investing,I remember seeing either on Reddit or somewher...,Stock Market Cycle Phrase,0.67,6.0,0.0,6.0,https://www.reddit.com/r/investing/comments/14...,[],"(NEGATIVE, 0.9741780161857605)"
917,t3_14aiciu,1.686876e+09,investing,My friend started a business which I recently ...,Is there a no cost way to have a 401k for a 2 ...,0.67,3.0,0.0,3.0,https://www.reddit.com/r/investing/comments/14...,[VOO],"(NEGATIVE, 0.998371422290802)"
918,t3_14aiafv,1.686875e+09,investing,(Un)Fortunately I came with some large cash wh...,Hesitant to invest during bull runs,0.27,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...,[],"(NEGATIVE, 0.9999454021453857)"
919,t3_14agk4s,1.686871e+09,investing,"Hello everyone,\n\nI'm 28 years old and would ...",Need some advice on different IRA's,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/14...,[],"(NEGATIVE, 0.9999575614929199)"
920,t3_14ag5ge,1.686870e+09,investing,Can someone explain to me what the best option...,"Income Taxes—What Options Are Best? CDs, Bonds...",0.50,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...,[APR],"(NEGATIVE, 0.9988001585006714)"
921,t3_14afr25,1.686869e+09,investing,"In 2017, my employer started an ESOP program a...",Can't cash out my employee stock until I'm 73?,0.50,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...,[ESOP],"(NEGATIVE, 0.9965392351150513)"
922,t3_14afqlk,1.686869e+09,investing,I'm using a source where it doesn't calculate...,Confused with the Adjusted Closing Price for C...,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/14...,"[YAHOO FINANCE, THE ADJUSTED CLOSING PRICE]","(NEGATIVE, 0.9995334148406982)"
923,t3_14af1r3,1.686867e+09,investing,Obviously this comes down to personal risk app...,Are emergency funds really necessary?,0.22,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...,[],"(NEGATIVE, 0.9998340606689453)"
924,t3_14adyi0,1.686864e+09,investing,"Hi there,\n\nI recently bought long-bonds when...",Bond outlook when CB drop rates?,0.60,1.0,0.0,1.0,https://www.reddit.com/r/investing/comments/14...,[],"(NEGATIVE, 0.9979787468910217)"
925,t3_14ad4a3,1.686862e+09,investing,An ETF that I hold Call Options on announced i...,What happens to Call Options in a Reverse Stoc...,0.33,0.0,0.0,0.0,https://www.reddit.com/r/investing/comments/14...,[CALL OPTIONS],"(NEGATIVE, 0.9998555183410645)"


##### Extract organization sentiment direction and score

In [41]:
sentiment = {}

for i, row in data.iterrows():
  direction = row['sentiment'][0]
  score = row['sentiment'][1]
  for org in row["organizations"]:
    sentiment[org] = sentiment.get(org, {"POSITIVE": [], "NEGATIVE": []})
    sentiment[org][direction].append(score)

In [42]:
sentiment["MSFT"] # Many negative comments about microsoft

{'POSITIVE': [0.9175806641578674],
 'NEGATIVE': [0.8897457122802734,
  0.9995661377906799,
  0.9999691247940063,
  0.9999969005584717,
  0.9995026588439941,
  0.9957656860351562,
  0.9997864365577698,
  0.9995001554489136]}

In [43]:
sentiment["VOO"]

{'POSITIVE': [0.7488009333610535,
  0.9351088404655457,
  0.5514634847640991,
  0.5898270010948181,
  0.9526062607765198,
  0.9979812502861023,
  0.9175806641578674,
  0.9442991614341736],
 'NEGATIVE': [0.9932045936584473,
  0.9959081411361694,
  0.99895179271698,
  0.9959806203842163,
  0.9993301630020142,
  0.5207668542861938,
  0.9999226331710815,
  0.9999942779541016,
  0.9999924898147583,
  0.996562659740448,
  0.9994623064994812,
  0.9825721979141235,
  0.999958872795105,
  0.6766064763069153,
  0.903938889503479,
  0.9995065927505493,
  0.9988573789596558,
  0.9990026354789734,
  0.9999834299087524,
  0.9995026588439941,
  0.9999620914459229,
  0.9994974136352539,
  0.9777384996414185,
  0.9388285875320435,
  0.8616629242897034,
  0.7812795639038086,
  0.9955328702926636,
  0.9957656860351562,
  0.9999852180480957,
  0.9939053654670715,
  0.9986045956611633,
  0.858549952507019,
  0.999578058719635,
  0.9973453879356384,
  0.9935573935508728,
  0.9994423985481262,
  0.9291248917

In [44]:
sentiment_ratio = []

for org in sentiment.keys():
  pos_ratio = np.nan_to_num(np.mean(sentiment[org]["POSITIVE"]))
  neg_ratio = np.nan_to_num(np.mean(sentiment[org]["NEGATIVE"]))
  avg_score = sum(sentiment[org]["POSITIVE"]) - sum(sentiment[org]["NEGATIVE"])
  freqs = len(sentiment[org]["POSITIVE"]) + len(sentiment[org]["NEGATIVE"])

  sentiment_ratio.append({
      "entity": org,
      "positive_ratio": pos_ratio,
      "negative_ratio": neg_ratio,
      "freqs": freqs,
      "score": avg_score / (freqs) if freqs != 0 else 0
  })

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [45]:
sentiment_ratio

[{'entity': 'WIKI',
  'positive_ratio': 0.0,
  'negative_ratio': 0.9986215829849243,
  'freqs': 46,
  'score': -0.9986215829849243},
 {'entity': 'THE BRICS SUMMIT',
  'positive_ratio': 0.7172220945358276,
  'negative_ratio': 0.0,
  'freqs': 1,
  'score': 0.7172220945358276},
 {'entity': 'VOO',
  'positive_ratio': 0.8297084495425224,
  'negative_ratio': 0.9538937003716178,
  'freqs': 54,
  'score': -0.6896563448287822},
 {'entity': 'STKB',
  'positive_ratio': 0.0,
  'negative_ratio': 0.9998903274536133,
  'freqs': 1,
  'score': -0.9998903274536133},
 {'entity': 'STKA',
  'positive_ratio': 0.0,
  'negative_ratio': 0.9998903274536133,
  'freqs': 1,
  'score': -0.9998903274536133},
 {'entity': 'MERRILL LYNCH',
  'positive_ratio': 0.0,
  'negative_ratio': 0.9803093671798706,
  'freqs': 2,
  'score': -0.9803093671798706},
 {'entity': 'CRASHCOURSE ECONOMICS',
  'positive_ratio': 0.0,
  'negative_ratio': 0.9997691512107849,
  'freqs': 1,
  'score': -0.9997691512107849},
 {'entity': 'BOGLE',
  

In [46]:
sentiment_df = pd.DataFrame(sentiment_ratio)

In [47]:
sentiment_df.head()

,entity,positive_ratio,negative_ratio,freqs,score
0,WIKI,0.000000,0.998622,46,-0.998622
1,THE BRICS SUMMIT,0.717222,0.000000,1,0.717222
2,VOO,0.829708,0.953894,54,-0.689656
3,STKB,0.000000,0.999890,1,-0.999890
4,STKA,0.000000,0.999890,1,-0.999890


In [49]:
sentiment_valid = sentiment_df.loc[sentiment_df["freqs"] > 3]

In [50]:
sentiment_valid.head(5)

,entity,positive_ratio,negative_ratio,freqs,score
0,WIKI,0.000000,0.998622,46,-0.998622
2,VOO,0.829708,0.953894,54,-0.689656
8,FIDELITY,0.854927,0.969063,46,-0.691499
10,NAV,0.000000,0.990258,5,-0.990258
12,META,0.980878,0.987275,9,-0.768591


### By Sorting the dataframe based on the score value in the descending order we can see the badly rated stock options first

In [52]:
sentiment_valid.sort_values('score', ascending=True)

,entity,positive_ratio,negative_ratio,freqs,score
0,WIKI,0.000000,0.998622,46,-0.998622
240,FDIC,0.000000,0.996937,6,-0.996937
16,MICROSOFT,0.000000,0.994343,6,-0.994343
10,NAV,0.000000,0.990258,5,-0.990258
80,FED,0.000000,0.927374,23,-0.927374
104,ROBINHOOD,0.000000,0.912318,4,-0.912318
521,MORGAN STANLEY,0.000000,0.887479,4,-0.887479
36,QQQ,0.917581,0.976583,17,-0.865161
24,APPLE,0.590151,0.994538,10,-0.836069
103,SPY,0.861083,0.996814,18,-0.790381
